In [1]:
import preprocessing as preproc
from utils import *
import numpy as np
import cPickle as pickle

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
/opt/conda/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [4]:
base_images_path = train_images_path
base_annotation_path = train_annotation_path

# Build Data - Data structure

In [5]:
image_data_arr = preproc.get_image_data_arr(base_images_path,base_annotation_path)
print(len(image_data_arr))

100%|██████████| 82783/82783 [18:17<00:00, 75.42it/s]

82612


# Build Data - Images & captions

In [6]:
images_concat_t = preproc.construct_images_concat_t(image_data_arr)
captions = preproc.get_captions_list(image_data_arr)[0]

print(images_concat_t.shape)
print(len(captions))

100%|██████████| 5/5 [00:00<00:00,  7.04it/s]

(82612, 3, 224, 224)
82612


# Filter Data

In [7]:
MIN_NO_OF_APP = 3
MAX_LENGTH = 15

# Common words filter

In [8]:
common_words_caption_mask = preproc.compute_common_words_caption_mask(captions,min_no_of_app = MIN_NO_OF_APP)
print("Before = %d \nAfter = %d"%(len(captions),np.sum(common_words_caption_mask)))

Before = 82612 
After = 71742


# Max caption lenght filter

In [9]:
max_length_mask = preproc.get_short_caption_mask(captions, MAX_LENGTH)
print("Before = %d \nAfter = %d"%(len(captions),np.sum(max_length_mask)))

Before = 82612 
After = 71495


# Combine masks

In [11]:
combined_masks = [a and b for a, b in zip(common_words_caption_mask, max_length_mask)]
print("Before all = %d \nAfter all = %d"%(len(captions),np.sum(combined_masks)))

Before all = 82612 
After all = 62735


In [12]:
captions_filtered = preproc.filter_array_by_mask(captions,combined_masks)
images_concat_t_filtered = preproc.filter_array_by_mask(images_concat_t,combined_masks)

In [13]:
print(images_concat_t_filtered.shape)
print(len(captions_filtered))

(62735, 3, 224, 224)
62735


# Write data to disk

In [14]:
write_path = app_3_length_15_data_path + train_folder
write_images_path = write_path + images_concat_folder
write_captions_path = write_path + captions_folder

In [17]:
save_array(write_images_path + 'images_concat.bc', images_concat_t_filtered)
pickle.dump(captions_filtered, open(write_captions_path + "captions_batch_0.p", "wb"))

# Build General Purpose Data Structures ( just once )

In [27]:
train_path = full_data_path + train_folder
val_path = full_data_path + val_folder

In [28]:
train_raw_captions = preproc.get_captions_from_batch(train_path + captions_folder, batch_nr = 0)
val_raw_captions = preproc.get_captions_from_batch(val_path  + captions_folder, batch_nr = 0)

In [30]:
all_raw_captions = list(train_raw_captions) + list(val_raw_captions)
len(all_raw_captions)

122050

In [31]:
unique_words = preproc.get_unique_words(all_raw_captions)
(word2index, index2word) = preproc.get_index_word_dicts(unique_words)
print(len(unique_words))

22261


In [32]:
preproc.save_obj(unique_words,full_data_path+general_datastruct_folder+"unique_words")
preproc.save_obj(word2index,full_data_path+general_datastruct_folder+"word2index")
preproc.save_obj(index2word,full_data_path+general_datastruct_folder+"index2word")